# Distributed Grover's Search
____


This python notebook is useful to exploit a distributed version of the<b> Grover's algorithm </b> implemented according the D-NISQ reference model proposed in the article "Distributed Noisy-Intermediate Scale Quantum: an Application to Grover-based Algorithms" by Giovanni Acampora, Ferdinando Di Martino, Alfredo Massa, Roberto Schiattarella, Autilia Vitiello submitted to the journal Information Fusion (Elsevier).
 
For more detail about the  Grover's Algorithn (GA) and its distributed version on the D-NISQ architecture, please, refer to the aforementioned paper.
 
Let us start importing the required libreries. 

In [ ]:
from threading import *
from concurrent.futures import *
from datetime import datetime
from statistics import mean

from GroverCircuitBuilder import *
from QuantumComputing import *
from DistributedGroverAlgorithm_1 import *
dateandtime = datetime.now().strftime("%d.%m.%Y  %H.%M.%S")

After importing the required libraries, it is necessary to set the backend to be used for running GA. 

Distributed GA can be executed both on simulators and real devices. Note, that for using real IBM quantum computers you need an access provider.

Please, set:
    
   - SIMULATION: True for simulating the algorithm, False for using a real quantum device.
    
   - DIFFERENT_BACKENDS: True to assign different backends to different sub-problems. False to use the same backend for all the sub-problems;

   - SHOTS: Number of shots in each backend;
    
   - RUN: number of executions of the algorithm;

As defult, in the case <i> SIMULATION = True </i> a fake backend will be used, so the simulation will take in account the noise of the quantum device. Please, specify the fake device you want to use in the BACKEND variable in SIMULATION PARAMETERS. Moreover, if you intend to simulate the algorithm without noise, you can set <i> NOISE = False </i> to use the 'qasm_simulator'.

On the other hand, if you want to use a real quantum device you have to set your IBM account details in the REAL EXECUTION PARAMETERS part of the following cell and pass in the BACKEND variable the name of the quantum device you are going to use as string. 
 

In [ ]:
# BACKEND SETTINGS ----------------------------------------------------------------------------------------------------

SIMULATION = True                       
DIFFERENT_BACKENDS = False 
SHOTS = 8192
RUN = 1


# SIMULATION PARAMETERS --------------------------------------------------------------- 
if SIMULATION:
    NOISE = True       
    BACKEND = FakeAlmaden()                  
    PROVIDER=None
# REAL EXECUTION PARAMETERS ---------------------------------------------------------------    
else:
    IBMQ.enable_account('INSERT TOKEN')
    PROVIDER = IBMQ.get_provider(hub='HUB', group='GROUP', project ='MAIN')
    BACKEND = 'ibmq_almaden'                  

Moreover, it is necessary some parameters to store results:

   - REPORT_FILE : set True to create a .txt log file;
   - FILE_PATH : specify the log file path
   - FILE_NAME : specify the log file name.

In [ ]:
# GENERAL SETTINGS ----------------------------------------------------------------------------------------------------
                
REPORT_FILE = False                     
FILE_PATH = f"C:\\"
FILE_NAME = f"Report_{dateandtime}.txt"

Provide informations about the database on which execute the Grover's Search:

   - N : database size. (It must be a power of 2);
   - WINNER : winner item. Please, note that it is an arbitrary variable;
   - OTHER : non-winner strings to fill the database. Also this variable is arbitrary;
   - WINNER_INDEX : location in the database of the winner item. (arbitrary).

  
  

In [ ]:
# DATABASE SETTINGS ---------------------------------------------------------------------------------------------------

N = 128                                    
WINNER = "1111111111"                    
OTHER = "0000000000"                     
WINNER_INDEX = 10                        


Set the parameters for the D-Nisq architecture:

   - NUMBER_OF_SUBPROBLEMS: the number of sub-problems created by the  Decomposition Layer 


In [ ]:
# PARALLELISM SETTINGS ------------------------------------------------------------------------------------------------

NUMBER_OF_SUBPROBLEMS = 32               # Set 1 to perform a standard search without parallelism


Finally, set the parameters for the Grover's algorithms:

   - MANUAL_ITERATIONS: an integer indicating the number of iterations of each Grover's algorithm
   - HISTOGRAMS: True to plot counts histograms at the end of each Grover search

In [ ]:
# GROVER'S ALGORITHM SETTINGS -----------------------------------------------------------------------------------------

MANUAL_ITERATIONS = 1                
HISTOGRAMS = False                       


Run the following cell to import the backend, build the database and the quantum circuit useful to implement the Grover's search in each thread. 

In [ ]:
# DISTRIBUTED GROVER'S SEARCH -----------------------------------------------------------------------------------------

max_times = []
successes = 0
sub_N = int(N / NUMBER_OF_SUBPROBLEMS)

assert N % NUMBER_OF_SUBPROBLEMS == 0, "Please choose the number of subproblems as a divisor of N."

sub_n = int(log(sub_N, 2))
THREADLOCK = Lock()

write_report(report_opening(dateandtime, RUN, N, NUMBER_OF_SUBPROBLEMS, sub_N, sub_n, MANUAL_ITERATIONS, SHOTS,
                           SIMULATION, NOISE, DIFFERENT_BACKENDS, BACKEND), REPORT_FILE, FILE_PATH + FILE_NAME)

if SIMULATION and NOISE:
    print("\nCreating a mock-backend list...")
    MOCK_BACKEND_LIST = [QasmSimulator.from_backend(BACKEND) for _ in range(NUMBER_OF_SUBPROBLEMS)]
    print("\nMock-backend list created.")
else:
    MOCK_BACKEND_LIST = None

if DIFFERENT_BACKENDS:
    backend_list, backend_information = ask_for_backends(NUMBER_OF_SUBPROBLEMS)
    write_report(backend_information, REPORT_FILE, FILE_PATH + FILE_NAME)

database = create_database(N, WINNER, OTHER, WINNER_INDEX)
sub_databases, winner_coordinates = create_sub_databases(database, NUMBER_OF_SUBPROBLEMS, sub_N, WINNER)
sub_basis = create_sub_basis(NUMBER_OF_SUBPROBLEMS, sub_N, sub_n)
sub_oracles = create_sub_oracles(NUMBER_OF_SUBPROBLEMS, sub_N, winner_coordinates)
sub_diffusers = create_sub_diffusers(NUMBER_OF_SUBPROBLEMS, sub_N, sub_basis)

sub_grover_circuits = [grover_circuit_builder(sub_N, sub_n, sub_oracles[i], sub_diffusers[i], MANUAL_ITERATIONS)
                       for i in range(NUMBER_OF_SUBPROBLEMS)]




Execute the Distributed Grover's Search for a number of runs indicated in the aforementioned variable RUN.

In [ ]:
for run in range(RUN):

    run_information = f"\n\n\n------------------------ RUN {run+1} -----------------------------\n"
    write_report(run_information, REPORT_FILE, FILE_PATH + FILE_NAME)                    
    results = distributed_grover_search(NUMBER_OF_SUBPROBLEMS, sub_grover_circuits, SIMULATION, NOISE, SHOTS, BACKEND,
                                        MOCK_BACKEND_LIST, PROVIDER)

    sub_counts, counts_information = get_counts(results, NUMBER_OF_SUBPROBLEMS)
    write_report(counts_information, REPORT_FILE, FILE_PATH + FILE_NAME)

    execution_times, max_time, times_information = get_times(results, NUMBER_OF_SUBPROBLEMS)
    max_times.append(max_time)
    write_report(times_information, REPORT_FILE, FILE_PATH + FILE_NAME)

    success, success_information = verify_success(NUMBER_OF_SUBPROBLEMS, sub_N, sub_n, sub_counts, WINNER_INDEX)
    write_report(success_information, REPORT_FILE, FILE_PATH + FILE_NAME)

    if success:
        successes += 1

    write_report(current_situation(run + 1, max_times, successes), REPORT_FILE, FILE_PATH + FILE_NAME)

    if HISTOGRAMS:
        show_data(NUMBER_OF_SUBPROBLEMS, sub_counts)

write_report("\n\n\n\n", REPORT_FILE, FILE_PATH + FILE_NAME)


In the output of the above cell, it can be seen the success rate of the Distributed Grover's Algorithm. Moreover, during the execution indications about the status of each thread and the counts obtaining at the end of each sub-search are given.

Now, a not distributed Grover's search can be conducted so as to obtain a comparison in terms of success rate with the distributed Grover's algorithm. 

To perform the standard Grover's search it has to be set the variable <i> NUMBER_OF_SUBPROBLEMS = 1</i>. Please, set the number of queries to the Grover's oracle as $O(\sqrt(N))$ to maximize the probability of find the right item in the database.

In [ ]:
NUMBER_OF_SUBPROBLEMS = 1
MANUAL_ITERATIONS = 8  


In [ ]:
 # STANDARD GROVER'S SEARCH -----------------------------------------------------------------------------------------

max_times = []
successes = 0
sub_N = int(N / NUMBER_OF_SUBPROBLEMS)

assert N % NUMBER_OF_SUBPROBLEMS == 0, "Please choose the number of subproblems as a divisor of N."

sub_n = int(log(sub_N, 2))
THREADLOCK = Lock()

write_report(report_opening(dateandtime, RUN, N, NUMBER_OF_SUBPROBLEMS, sub_N, sub_n, MANUAL_ITERATIONS, SHOTS,
                           SIMULATION, NOISE, DIFFERENT_BACKENDS, BACKEND), REPORT_FILE, FILE_PATH + FILE_NAME)

if SIMULATION and NOISE:
    print("\nCreating a mock-backend list...")
    MOCK_BACKEND_LIST = [QasmSimulator.from_backend(BACKEND) for _ in range(NUMBER_OF_SUBPROBLEMS)]
    print("\nMock-backend list created.")
else:
    MOCK_BACKEND_LIST = None

if DIFFERENT_BACKENDS:
    backend_list, backend_information = ask_for_backends(NUMBER_OF_SUBPROBLEMS)
    write_report(backend_information, REPORT_FILE, FILE_PATH + FILE_NAME)

database = create_database(N, WINNER, OTHER, WINNER_INDEX)
sub_databases, winner_coordinates = create_sub_databases(database, NUMBER_OF_SUBPROBLEMS, sub_N, WINNER)
sub_basis = create_sub_basis(NUMBER_OF_SUBPROBLEMS, sub_N, sub_n)
sub_oracles = create_sub_oracles(NUMBER_OF_SUBPROBLEMS, sub_N, winner_coordinates)
sub_diffusers = create_sub_diffusers(NUMBER_OF_SUBPROBLEMS, sub_N, sub_basis)

sub_grover_circuits = [grover_circuit_builder(sub_N, sub_n, sub_oracles[i], sub_diffusers[i], MANUAL_ITERATIONS)
                       for i in range(NUMBER_OF_SUBPROBLEMS)]





Finally, you can run the following cell to execute the standard Grover's search and check for the results.

In [ ]:
for run in range(RUN):

    run_information = f"\n\n\n------------------------ RUN {run+1} -----------------------------\n"
    write_report(run_information, REPORT_FILE, FILE_PATH + FILE_NAME)                    
    results = distributed_grover_search(NUMBER_OF_SUBPROBLEMS, sub_grover_circuits, SIMULATION, NOISE, SHOTS, BACKEND,
                                        MOCK_BACKEND_LIST, PROVIDER)

    sub_counts, counts_information = get_counts(results, NUMBER_OF_SUBPROBLEMS)
    write_report(counts_information, REPORT_FILE, FILE_PATH + FILE_NAME)

    execution_times, max_time, times_information = get_times(results, NUMBER_OF_SUBPROBLEMS)
    max_times.append(max_time)
    write_report(times_information, REPORT_FILE, FILE_PATH + FILE_NAME)

    success, success_information = verify_success(NUMBER_OF_SUBPROBLEMS, sub_N, sub_n, sub_counts, WINNER_INDEX)
    write_report(success_information, REPORT_FILE, FILE_PATH + FILE_NAME)

    if success:
        successes += 1

    write_report(current_situation(run + 1, max_times, successes), REPORT_FILE, FILE_PATH + FILE_NAME)

    if HISTOGRAMS:
        show_data(NUMBER_OF_SUBPROBLEMS, sub_counts)

write_report("\n\n\n\n", REPORT_FILE, FILE_PATH + FILE_NAME)
